# Deploying Machine Learning Models using ksonnet and Ambassador
## Experimental


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

## Create Namespace

In [4]:
!kubectl create namespace seldon

namespace "seldon" created


## Install Ambassador
See the Abassador [getting started](https://www.getambassador.io/user-guide/getting-started) docs. Eventually, this would also be done via ksonnet.

In [17]:
!kubectl apply -f resources/ambassador-service-minikube.yaml -n seldon

service "ambassador" created


In [18]:
!kubectl apply -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml -n seldon

service "ambassador-admin" created
deployment "ambassador" created


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [19]:
!ks init my-ml-deployment --api-spec=version:v1.8.0

INFO  Using context 'minikube' from the kubeconfig file specified at the environment variable $KUBECONFIG
INFO  Creating a new app 'my-ml-deployment' at path '/home/clive/work/seldon/fork-seldon-core/notebooks/my-ml-deployment'
WARNING !
Kubernetes version v1.8.0 is currently supported as Beta; you may encounter unexpected behavior
INFO  Creating environment 'default' with namespace 'seldon', pointing at server at address 'https://192.168.99.100:8443'
INFO  Generating environment metadata at path '/home/clive/work/seldon/fork-seldon-core/notebooks/my-ml-deployment/environments/default'
INFO  ksonnet app successfully created! Next, try creating a component with `ks generate`.


In [20]:
!cd my-ml-deployment && \
    ks registry add seldon-core github.com/SeldonIO/seldon-core/tree/master/seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --apifeServiceType=NodePort --withApife=false --namespace=seldon

INFO  Retrieved 12 files
INFO  Writing component at 'components/seldon-core'


In [21]:
!cd my-ml-deployment && \
      ks apply default

INFO  Updating deployments seldon.seldon-cluster-manager
INFO  Creating non-existent deployments seldon.seldon-cluster-manager
INFO  Updating deployments seldon.redis
INFO  Creating non-existent deployments seldon.redis
INFO  Updating services seldon.redis
INFO  Creating non-existent services seldon.redis
INFO  Updating customresourcedefinitions seldondeployments.machinelearning.seldon.io


## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [22]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [31]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

NAMESPACE='seldon'
MINIKUBE_IP=commands.getoutput('minikube ip')
AMBASSADOR_PORT=commands.getoutput("kubectl get svc -n "+NAMESPACE+" -l service=ambassador -o jsonpath='{.items[0].spec.ports[0].nodePort}'")

def rest_request(deploymentName):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+AMBASSADOR_PORT+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload)
    print response.status_code
    print response.text   
    
    
def rest_request_auth(deploymentName,username,password):
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":"+AMBASSADOR_PORT+"/seldon/"+deploymentName+"/api/v0.1/predictions",
                json=payload,
                auth=HTTPBasicAuth(username, password))
    print response.status_code
    print response.text   

def grpc_request(deploymentName):
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":"+AMBASSADOR_PORT)
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('seldon',deploymentName)]
    response = stub.Predict(request=request,metadata=metadata)
    print response

## Create Seldon Deployment

**Check everything is running before continuing**

In [24]:
!kubectl get pods -n seldon

NAME                                                   READY     STATUS              RESTARTS   AGE
ambassador-5c785b76f7-9wfxl                            2/2       Running             0          3m
ambassador-5c785b76f7-cqszt                            2/2       Running             0          3m
ambassador-5c785b76f7-ftfdj                            2/2       Running             0          3m
redis-df886d999-4zgqm                                  1/1       Running             0          35s
seldon-cluster-manager-764c84b5bf-6m9cz                1/1       Running             0          35s
test-deployment-fx-market-predictor-779b869d5b-62sqz   0/2       ContainerCreating   0          19s


In [25]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-deployment-example" created


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [28]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [29]:
rest_request("seldon-deployment-example")

200
{
  "meta": {
    "puid": "f371jv27kmnl8btstc55h8llhp",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [32]:
grpc_request("seldon-deployment-example")

meta {
  puid: "aqfciqo2fke2deu60r02u6vp5a"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [33]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" created
deployment "example-auth" created


** Need to wait until running before adding Ambassador config **

In [35]:
!kubectl get pods -n seldon

NAME                                                 READY     STATUS    RESTARTS   AGE
ambassador-5c785b76f7-9wfxl                          2/2       Running   0          9m
ambassador-5c785b76f7-cqszt                          2/2       Running   0          9m
ambassador-5c785b76f7-ftfdj                          2/2       Running   0          9m
example-auth-54df78c744-c7sbz                        1/1       Running   0          38s
redis-df886d999-4zgqm                                1/1       Running   0          6m
seldon-cluster-manager-764c84b5bf-6m9cz              1/1       Running   0          6m
test-deployment-fx-market-predictor-5845c9cb-lxf52   2/2       Running   0          2m


In [36]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service "example-auth" configured


Show failed request when auth is running

In [37]:
rest_request("seldon-deployment-example")

401



Show successful request with auth

In [38]:
rest_request_auth("seldon-deployment-example","username","password")

200
{
  "meta": {
    "puid": "j5e9umnsmpg7nd0jo921hvlao4",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


# Tear down

In [39]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" deleted
deployment "example-auth" deleted


In [40]:
!kubectl delete -f https://getambassador.io/yaml/ambassador/ambassador-no-rbac.yaml -n seldon

service "ambassador-admin" deleted
deployment "ambassador" deleted


In [41]:
!kubectl delete -f resources/model.json

seldondeployment "seldon-deployment-example" deleted


In [42]:
!cd my-ml-deployment && ks delete default

ERROR strconv.Atoi: parsing "": invalid syntax


In [43]:
!rm -rf my-ml-deployment